# Exploratory Spatial and Temporal Data Analysis (ESTDA)



In [ ]:
import matplotlib
import numpy as np
import pysal as ps
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
f = ps.open(ps.examples.get_path('usjoin.csv'), 'r')

To determine what is in the file, check the `header` attribute on the file object:

In [ ]:
f.header[0:10]

Ok, lets pull in the `name` variable to see what we have.

In [ ]:
name = f.by_col('Name')

In [ ]:
name

Now obtain per capital incomes in 1929 which is in the column associated with `1929`.

In [ ]:
y1929 = f.by_col('1929')

In [ ]:
y1929[:10]

And now 2009

In [ ]:
y2009 = f.by_col("2009")

In [ ]:
y2009[:10]

These are read into regular Python lists which are not particularly well suited to efficient data analysis. So let's convert them to numpy arrays.

In [ ]:
y2009 = np.array(y2009)

In [ ]:
y2009

Much better. But pulling these in and converting them a column at a time is tedious and error prone. So we will do all of this in a list comprehension.

In [ ]:
Y = np.array( [ f.by_col(str(year)) for year in range(1929,2010) ] ) * 1.0

In [ ]:
Y.shape

In [ ]:
Y = Y.transpose()

In [ ]:
Y.shape

In [ ]:
years = np.arange(1929,2010)

In [ ]:
plt.plot(years,Y[0])

In [ ]:
RY = Y / Y.mean(axis=0)

In [ ]:
plt.plot(years,RY[0])

In [ ]:
name = np.array(name)

In [ ]:
np.nonzero(name=='Ohio')

In [ ]:
plt.plot(years, RY[32], label='Ohio')
plt.plot(years, RY[0], label='Alabama')
plt.legend()

## Spaghetti Plot

In [ ]:
for row in RY:
    plt.plot(years, row)

## Kernel Density (univariate, aspatial)

In [ ]:
from scipy.stats.kde import gaussian_kde

In [ ]:
density = gaussian_kde(Y[:,0])

In [ ]:
Y[:,0]

In [ ]:
density = gaussian_kde(Y[:,0])

In [ ]:
minY0 = Y[:,0].min()*.90
maxY0 = Y[:,0].max()*1.10
x = np.linspace(minY0, maxY0, 100)

In [ ]:
plt.plot(x,density(x))

In [ ]:
d2009 = gaussian_kde(Y[:,-1])

In [ ]:
minY0 = Y[:,-1].min()*.90
maxY0 = Y[:,-1].max()*1.10
x = np.linspace(minY0, maxY0, 100)

In [ ]:
plt.plot(x,d2009(x))

In [ ]:
minR0 = RY.min()

In [ ]:
maxR0 = RY.max()

In [ ]:
x = np.linspace(minR0, maxR0, 100)

In [ ]:
d1929 = gaussian_kde(RY[:,0])

In [ ]:
d2009 = gaussian_kde(RY[:,-1])

In [ ]:
plt.plot(x, d1929(x))
plt.plot(x, d2009(x))

In [ ]:
plt.plot(x, d1929(x), label='1929')
plt.plot(x, d2009(x), label='2009')
plt.legend()

In [ ]:
import seaborn as sns
for y in range(2010-1929):
    sns.kdeplot(RY[:,y])
#sns.kdeplot(data.HR80)
#sns.kdeplot(data.HR70)
#sns.kdeplot(data.HR60)


In [ ]:
import seaborn as sns
for y in range(2010-1929):
    sns.kdeplot(RY[:,y])

In [ ]:
for cs in RY.T: # take cross sections
    plt.plot(x, gaussian_kde(cs)(x))

In [ ]:
cs[0]

In [ ]:
sigma = RY.std(axis=0)
plt.plot(years, sigma)
plt.ylabel('s')
plt.xlabel('year')
plt.title("Sigma-Convergence")

So the distribution is becoming less dispersed over time.

But what about internal mixing? Do poor (rich) states remain poor (rich), or is there movement within the distribuiton over time?

## Markov Chains

In [ ]:
c = np.array([
['b','a','c'],
['c','c','a'],
['c','b','c'],
['a','a','b'],
['a','b','c']])

In [ ]:
c

In [ ]:
m = ps.Markov(c)

In [ ]:
m.classes

In [ ]:
m.transitions

In [ ]:
m.p

### State Per Capita Incomes

In [ ]:
ps.examples.explain('us_income')

In [ ]:
data = ps.pdio.read_files(ps.examples.get_path("us48.dbf"))
W = ps.queen_from_shapefile(ps.examples.get_path("us48.shp"))
W.transform = 'r'

In [ ]:
data.STATE_NAME

In [ ]:
f = ps.open(ps.examples.get_path("usjoin.csv"))
pci = np.array([f.by_col[str(y)] for y in range(1929,2010)])
pci.shape

In [ ]:
pci = pci.T

In [ ]:
pci.shape

In [ ]:
cnames = f.by_col('Name')

In [ ]:
cnames[:10]

In [ ]:
ids = [ cnames.index(name) for name in data.STATE_NAME]

In [ ]:
ids[:10]

In [ ]:
pci = pci[ids]
RY = RY[ids]

In [ ]:
import matplotlib.pyplot as plt

import geopandas as gpd
shp_link = ps.examples.get_path('us48.shp')
tx = gpd.read_file(shp_link)
pci29 = ps.Quantiles(pci[:,0], k=5)
f, ax = plt.subplots(1, figsize=(10, 5))
tx.assign(cl=pci29.yb+1).plot(column='cl', categorical=True, \
        k=5, cmap='Greens', linewidth=0.1, ax=ax, \
        edgecolor='grey', legend=True)
ax.set_axis_off()
plt.title('Per Capita Income 1929 Quintiles')

plt.show()

In [ ]:
pci2009 = ps.Quantiles(pci[:,-1], k=5)
f, ax = plt.subplots(1, figsize=(10, 5))
tx.assign(cl=pci2009.yb+1).plot(column='cl', categorical=True, \
        k=5, cmap='Greens', linewidth=0.1, ax=ax, \
        edgecolor='grey', legend=True)
ax.set_axis_off()
plt.title('Per Capita Income 2009 Quintiles')
plt.show()

## convert to a code cell to generate a time series of the maps
for y in range(2010-1929):
    pciy = ps.Quantiles(pci[:,y], k=5)
    f, ax = plt.subplots(1, figsize=(10, 5))
    tx.assign(cl=pciy.yb+1).plot(column='cl', categorical=True, \
            k=5, cmap='Greens', linewidth=0.1, ax=ax, \
            edgecolor='grey', legend=True)
    ax.set_axis_off()
    plt.title("Per Capita Income %d Quintiles"%(1929+y))
    plt.show()


Put series into cross-sectional quintiles (i.e., quintiles for each year).

In [ ]:
q5 = np.array([ps.Quantiles(y).yb for y in pci.T]).transpose()

In [ ]:
q5.shape

In [ ]:
q5[:,0]

In [ ]:
pci.shape

In [ ]:
pci[0]

we are looping over the rows of y which is ordered $T \times n$ (rows are cross sections, row 0 is the cross-section for period 0.

In [ ]:
m5 = ps.Markov(q5)

In [ ]:
m5.classes

In [ ]:
m5.transitions

In [ ]:
np.set_printoptions(3, suppress=True)
m5.p

In [ ]:
m5.steady_state #steady state distribution

In [ ]:
fmpt = ps.ergodic.fmpt(m5.p) #first mean passage time
fmpt

For a state with income in the first quintile, it takes on average 11.5 years for it to first enter the second quintile, 29.6 to get to the third quintile, 53.4 years to enter the fourth, and 103.6 years to reach the richest quintile.

But, this approach assumes the movement of a state in the income distribution is independent of the movement of its neighbors or the position of the neighbors in the distribution. Does spatial context matter?

## Dynamics of Spatial Dependence

Create a queen contiguity matrix that is row standardized

In [ ]:
w = ps.queen_from_shapefile(ps.examples.get_path('us48.shp'))
w.transform = 'R'

In [ ]:
mits = [ps.Moran(cs, w) for cs in RY.T]

In [ ]:
res = np.array([(m.I, m.EI, m.p_sim, m.z_sim) for m in mits])

In [ ]:
plt.plot(years, res[:,0], label='I')
plt.plot(years, res[:,1], label='E[I]')
plt.title("Moran's I")
plt.legend()

In [ ]:
plt.plot(years, res[:,-1])
plt.ylim(0,7.0)
plt.title('z-values, I')

## Spatial Markov

In [ ]:
pci.shape

In [ ]:
rpci = pci / pci.mean(axis=0)

In [ ]:
rpci[:,0]

In [ ]:
rpci[:,0].mean()

In [ ]:
sm = ps.Spatial_Markov(rpci, W, fixed=True, k=5)

In [ ]:
sm.p

In [ ]:
for p in sm.P:
    print(p)

In [ ]:
sm.S

In [ ]:
for f in sm.F:
    print(f)

In [ ]:
sm.summary()